# Read in data

In [1]:
from datetime import timedelta
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

airports = [
    "KATL",
    "KCLT",
    "KDEN",
    "KDFW",
    "KJFK",
    "KMEM",
    "KMIA",
    "KORD",
    "KPHX",
    "KSEA",
]

In [9]:
airport = "KDEN"

In [10]:
DATA_DIRECTORY = Path("code execution development data")

## LAMP

In [12]:
lamp = pd.read_csv(
    DATA_DIRECTORY / airport / f"{airport}_lamp.csv.bz2",
    parse_dates=["timestamp", "forecast_timestamp"],
)
lamp.head(5)

,timestamp,forecast_timestamp,temperature,wind_direction,wind_speed,wind_gust,cloud_ceiling,visibility,cloud,lightning_prob,precip
0,2020-11-14 00:30:00,2020-11-14 01:00:00,50,24,8,0,8.0,7,FW,N,False
1,2020-11-14 00:30:00,2020-11-14 02:00:00,47,23,9,0,8.0,7,FW,N,False
2,2020-11-14 00:30:00,2020-11-14 03:00:00,45,22,13,0,8.0,7,CL,N,False
3,2020-11-14 00:30:00,2020-11-14 04:00:00,43,22,15,0,8.0,7,FW,N,False
4,2020-11-14 00:30:00,2020-11-14 05:00:00,43,22,16,0,8.0,7,FW,N,False


## TBFM

In [13]:
tbfm = pd.read_csv(
    DATA_DIRECTORY / airport / f"{airport}_tbfm.csv.bz2",
    parse_dates=["timestamp", "scheduled_runway_estimated_time"],
)
tbfm.head(5)

,gufi,timestamp,scheduled_runway_estimated_time
0,UAL539.CLE.DEN.201112.2153.0048.TFM,2020-11-14,2020-11-14 01:15:07
1,SWA1002.MKE.DEN.201112.2313.0088.TFM,2020-11-14,2020-11-14 01:42:32
2,UAL1077.EWR.DEN.201112.2108.0106.TFM,2020-11-14,2020-11-14 01:22:02
3,SKW4752.CID.DEN.201112.2253.0165.TFM,2020-11-14,2020-11-14 01:10:54
4,SKW5234.GRR.DEN.201112.2218.0098.TFM,2020-11-14,2020-11-14 01:12:19


## ETD

In [11]:
etd = pd.read_csv(
    DATA_DIRECTORY / airport / f"{airport}_etd.csv.bz2",
    parse_dates=["departure_runway_estimated_time", "timestamp"],
)
etd.head(5)

,gufi,timestamp,departure_runway_estimated_time
0,SKW4678.DEN.FLG.201112.1843.0129.TFM,2020-11-13 18:00:05,2020-11-13 19:02:00
1,SKW4678.DEN.FLG.201112.1843.0129.TFM,2020-11-13 18:00:09,2020-11-13 19:07:00
2,SKW4678.DEN.FLG.201112.1843.0129.TFM,2020-11-13 18:00:10,2020-11-13 19:02:00
3,UAL459.DEN.MSP.201112.1638.0042.TFM,2020-11-13 18:00:14,2020-11-13 16:46:00
4,SWA6755.DEN.SMF.201112.1703.0087.TFM,2020-11-13 18:00:17,2020-11-13 17:26:00


## MFS

In [14]:
# TODO

# Filter Data

In [16]:
submission_format = pd.read_csv(
    DATA_DIRECTORY / "submission_format.csv", parse_dates=["timestamp"]
)

In [17]:
row = submission_format.iloc[545]
row

gufi                      DAL1568.DEN.SLC.201114.1257.0100.TFM
timestamp                                  2020-11-15 12:00:00
airport                                                   KDEN
minutes_until_pushback                                       0
Name: 545, dtype: object

In [18]:
current_time = row.timestamp
current_time

Timestamp('2020-11-15 12:00:00')

In [20]:
valid_time = current_time - timedelta(hours=30)
valid_time

Timestamp('2020-11-14 06:00:00')

## Filtered LAMP

In [32]:
valid_lamp = lamp.loc[(lamp.timestamp <= current_time) 
                    & (lamp.timestamp > valid_time) 
                    & (lamp.forecast_timestamp <= current_time) 
                    & (lamp.forecast_timestamp > valid_time)]
valid_lamp.iloc[-1]

timestamp             2020-11-15 11:30:00
forecast_timestamp    2020-11-15 12:00:00
temperature                            22
wind_direction                         16
wind_speed                              9
wind_gust                               0
cloud_ceiling                         8.0
visibility                              7
cloud                                  FW
lightning_prob                          N
precip                              False
Name: 1025, dtype: object

## Filtered TBFM

In [40]:
valid_tbfm = tbfm.loc[(tbfm.timestamp <= current_time) 
                    & (tbfm.timestamp > valid_time) ]
valid_tbfm['gufi'].nunique()

559

## Filtered ETD

In [27]:
valid_etd = etd.loc[(etd.timestamp <= current_time) 
                    & (etd.timestamp > valid_time) 
                    & (etd.gufi == row.gufi)]
valid_etd.iloc[-1]

gufi                               DAL1568.DEN.SLC.201114.1257.0100.TFM
timestamp                                           2020-11-15 10:34:03
departure_runway_estimated_time                     2020-11-15 13:26:00
Name: 16466, dtype: object